# **Visualización de datos**

**Asignatura:** Visualización de datos

**Autora:** Sandra Millán Palacios

**Profesor:** Rodrigo Díaz Morón



Para este proyecto de análisis y visualización de datos de ventas, realizaremos un análisis exhaustivo que nos permita explorar y entender el comportamiento de las ventas de un conjunto de datos que comprende transacciones realizadas en distintos países, a través de varias submarcas (que forman parte de la marca PepsiCo en Europa) y en diferentes periodos de tiempo. Para ello nos centraremos en explorar cómo se distribuyen y evolucionan las ventas de acuerdo a varias dimensiones clave:

1. Cómo se distribuyen las ventas realizadas en:

  - Cada país
  - Cada mes y año
  - Cada marca

2. Cuál es la tendencia y estacionalidad de:

  - Todas las ventas del país con menos ventas
  - La marca con más ventas

3. Cuáles son las predicciones hechas en España y cómo de
buenas son

## **Carga la base de datos**

In [477]:
pip install plotly_express

In [478]:
# Librerías necesarias

import plotly_express as px
import plotly.subplots as sp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
from google.colab import files
from scipy.stats import linregress
import numpy as np

In [479]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

La base de datos proporcionada consta de 18,666 registros y 8 columnas, cada una con un propósito específico relacionado con el seguimiento de ventas en distintos países, marcas, y en un periodo de tiempo detallado:

- **COUNTRY** - Indica el país donde se realizó la venta.
- **SUBBRAND** - Especifica la sub-marca vendida.
- **YEAR** - Año en que se realizó la venta o en que se realizó una predicción.
- **MONTH** - Mes en el que ocurrió la venta o la predicción.
- **SCENARIO** - Describe si el registro corresponde a una predicción o a una observación real.
- **FORECAST** - Mes en el que se realizó la predicción.
- **FORECAST_YEAR** - Año en el que se realizó la predicción.
- **AMOUNT** - Representa el volumen de ventas en una transacción. Es la métrica principal para evaluar el rendimiento de ventas, tendencias y precisión de pronósticos.

In [480]:
# Cargar la base de datos: datos_ejercicio_ventas.csv

uploaded = files.upload()

Saving datos_ejercicio_ventas.csv to datos_ejercicio_ventas (6).csv


In [481]:
df = pd.read_csv('/content/datos_ejercicio_ventas.csv')
df.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


## **Transformación de los datos**

Para comenzar el análisis de esta base de datos de ventas, primero dividiremos los registros en dos subconjuntos: uno que contenga únicamente los datos de ventas reales y otro que incluya las predicciones. Esta división facilitará un estudio detallado y específico de cada grupo de datos.

### **Crear dos subconjuntos de datos**

Crear dos dataset: uno de actuals (real) y el otro de Al_forecast (predicción)

In [482]:
df_actual = df[df['SCENARIO'] == 'actual']
df_forecast = df[df['SCENARIO'] == 'AI_forecast']

In [483]:
df.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


Eliminamos la columna FORECAST Y FORECAST_YEAR de los datos reales, ya que no hya ningún mes de predicción.

In [484]:
df_actual = df_actual.drop(['FORECAST', 'FORECAST_YEAR'], axis=1)

Vuelvo a reiniciar los indices para mayor consistencia de los datos

In [485]:
df_actual.reset_index(drop=True, inplace=True)
df_forecast.reset_index(drop=True, inplace=True)

## **Número de submarcas y países**

En total hay 9 países y 6 submarcas. En el siguiente gráfico no solo hemso mostrado el número de paises y de submarcas, sino que hemos visto si hay una cantidad de datos similares para cada representación. Podemos observar, que tenmso muy poco registros de Italia y de Mountain Dew.

In [486]:
def plot_scenario_counts(df, col='COUNTRY', scenario_col='SCENARIO'):
    # Contar la cantidad de registros por combinación de COUNTRY y SCENARIO
    df_counts = df.groupby([col, scenario_col]).size().reset_index(name="Count")

    # Crear el gráfico de barras con el número encima de cada barra
    fig = px.bar(
        df_counts,
        x=col,
        y="Count",
        color=scenario_col,
        barmode="group",
        text="Count",  # Mostrar el valor encima de cada barra
        title=f"Cantidad de Registros por {col} y Escenario"
    )

    # Ajustar el tamaño del texto
    fig.update_traces(texttemplate='%{text}', textposition='outside', textfont_size=10)

    # Mostrar el gráfico
    fig.show()

In [487]:
plot_scenario_counts(df, 'COUNTRY', 'SCENARIO')

In [488]:
plot_scenario_counts(df, 'SUBBRAND', 'SCENARIO')

Para visualizar mejor los datos reales y facilitar la identificación de aquellos países con mayor o menor cantidad de datos, se ha creado una nueva gráfica que muestra exclusivamente los datos reales, excluyendo las predicciones. Además, se representará el porcentaje que cada país representa del total de datos reales, proporcionando una visión más precisa.

In [489]:
# Calcular el total de datos en el DataFrame
total_datos = len(df_actual)

# Calcular el porcentaje de cada 'COUNTRY' y 'SUBBRAND' sobre el total de datos
df_country_pct = df_actual['COUNTRY'].value_counts(normalize=True).reset_index()
df_country_pct.columns = ['COUNTRY', 'PCT_COUNTRY']
df_country_pct['PCT_COUNTRY'] *= 100  # Convertir a porcentaje

df_subbrand_pct = df_actual['SUBBRAND'].value_counts(normalize=True).reset_index()
df_subbrand_pct.columns = ['SUBBRAND', 'PCT_SUBBRAND']
df_subbrand_pct['PCT_SUBBRAND'] *= 100  # Convertir a porcentaje

# Crear el gráfico de barras para 'COUNTRY' con porcentajes
fig_country = px.bar(df_country_pct, x='COUNTRY', y='PCT_COUNTRY', title='Porcentaje de Datos - Country', labels={'COUNTRY': 'Country', 'PCT_COUNTRY': 'Porcentaje de Datos'})
fig_country.update_traces(text=df_country_pct['PCT_COUNTRY'].round(2).astype(str) + '%', textposition='outside')

# Crear el gráfico de barras para 'SUBBRAND' con porcentajes
fig_subbrand = px.bar(df_subbrand_pct, x='SUBBRAND', y='PCT_SUBBRAND', title='Porcentaje de Datos - Subbrand', labels={'SUBBRAND': 'Subbrand', 'PCT_SUBBRAND': 'Porcentaje de Datos'})
fig_subbrand.update_traces(text=df_subbrand_pct['PCT_SUBBRAND'].round(2).astype(str) + '%', textposition='outside')

# Crear un layout de subplots y agregar ambos gráficos
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Porcentaje de Datos - Country", "Porcentaje de Datos - Subbrand"))

# Añadir gráficos al layout
fig.add_traces(fig_country['data'], rows=1, cols=1)
fig.add_traces(fig_subbrand['data'], rows=1, cols=2)

# Ajustar el diseño y mostrar
fig.update_layout(title_text="Porcentaje de Datos Reales por Pais y Submarca", showlegend=False)
fig.show()

## **Análisis de datos reales**

Vamos a analizar si los datos son consistentes o si, en cambio, puede haber algún tipo de error en ellos. Observamos que existen cantidades negativas, lo cual solo tendría sentido si representaran pérdidas. A continuación, examinaremos cuántos datos presentan valores negativos y evaluaremos si tienen coherencia en el contexto de los datos.

In [490]:
df_actual[['YEAR', 'MONTH', 'AMOUNT']].describe()

,YEAR,MONTH,AMOUNT
count,900.000000,900.000000,9.000000e+02
mean,2023.402222,5.697778,7.443485e+05
std,0.490619,3.194440,1.905511e+06
min,2023.000000,1.000000,-2.171201e+05
25%,2023.000000,3.000000,5.574319e+04
50%,2023.000000,5.500000,1.856788e+05
75%,2024.000000,8.000000,6.432082e+05
max,2024.000000,12.000000,1.481563e+07


**Vamos a crear la columna Fecha**

Para poder visulizar correctamente las series temporales.

In [491]:
df_actual['Fecha'] = pd.to_datetime(df_actual[['YEAR', 'MONTH']].assign(DAY=1))
df_actual = df_actual.sort_values(by="Fecha")

### **Datos negativos**

Vemos si los datos son coherentes

In [492]:
df_actual[df_actual['AMOUNT'] < 100]

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,AMOUNT,Fecha
539,Hungary,7up (L3),2023,7,actual,-6332.708713,2023-07-01
569,Norway,Lipton (L3),2023,9,actual,-217120.103133,2023-09-01
317,Hungary,7up (L3),2023,9,actual,-19481.652378,2023-09-01
417,Hungary,7up (L3),2023,11,actual,-13549.371551,2023-11-01
682,Hungary,7up (L3),2023,12,actual,-10.791926,2023-12-01
363,Hungary,7up (L3),2024,1,actual,-188.474017,2024-01-01
823,Hungary,7up (L3),2024,2,actual,-1.394871,2024-02-01
610,Norway,Lipton (L3),2024,2,actual,73.968144,2024-02-01
165,Czech,7up Free (L3),2024,2,actual,89.354912,2024-02-01
750,Norway,Lipton (L3),2024,3,actual,-173.296795,2024-03-01


A continuación, se procede a visualizar los datos de 7up en Hungría y los datos de Lipton en Noruega. Al revisar estos datos, se observó que algunos valores de ventas aparecen como negativos. Dado que las ventas se miden en litros, es imposible que estas cantidades sean negativas, ya que no tiene sentido vender "litros negativos" de producto. Este tipo de valores no es válido en el contexto de análisis y puede llevar a interpretaciones erróneas. Por esta razón, se procederá a eliminar los datos negativos antes de realizar la visualización, asegurando que el análisis refleje únicamente información coherente y útil.

In [493]:
# Filtrar los datos de 7up en Hungría
df_7up_hungary = df_actual[(df_actual["SUBBRAND"] == "7up (L3)") & (df_actual["COUNTRY"] == "Hungary")]

# Filtrar los datos de Lipton en Noruega
df_lipton_norway = df_actual[(df_actual["SUBBRAND"] == "Lipton (L3)") & (df_actual["COUNTRY"] == "Norway")]

# Convertir las fechas para mostrar solo el mes y el año
df_7up_hungary["Fecha"] = pd.to_datetime(df_7up_hungary["Fecha"]).dt.to_period("M").astype(str)
df_lipton_norway["Fecha"] = pd.to_datetime(df_lipton_norway["Fecha"]).dt.to_period("M").astype(str)

# Crear subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=("7up en Hungría", "Lipton en Noruega"))

# Especificar el color para ambas líneas
line_color = 'blue'  # Puedes cambiar este color a tu preferencia

# Agregar la línea de 7up en Hungría con el color especificado
fig.add_trace(
    go.Scatter(x=df_7up_hungary["Fecha"], y=df_7up_hungary["AMOUNT"], mode='lines', name="7up en Hungría", line=dict(color=line_color)),
    row=1, col=1
)

# Agregar la línea de Lipton en Noruega con el mismo color
fig.add_trace(
    go.Scatter(x=df_lipton_norway["Fecha"], y=df_lipton_norway["AMOUNT"], mode='lines', name="Lipton en Noruega", line=dict(color=line_color)),
    row=1, col=2
)

# Configurar los títulos y etiquetas de los ejes
fig.update_xaxes(title_text="Mes y Año", tickangle=-45, row=1, col=1)
fig.update_yaxes(title_text="Ventas", row=1, col=1)
fig.update_xaxes(title_text="Mes y Año", tickangle=-45, row=1, col=2)
fig.update_yaxes(title_text="Ventas", row=1, col=2)

# Configurar el diseño general
fig.update_layout(title_text="Ventas de 7up en Hungría y Lipton en Noruega", showlegend=False)

# Mostrar la figura
fig.show()

<ipython-input-493-d8fed143d66e>:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-493-d8fed143d66e>:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [494]:
# Eliminar registros de 7up (L3) en Hungría y Lipton (L3) en Noruega
df_actual = df_actual[~((df_actual['SUBBRAND'] == '7up (L3)') & (df_actual['COUNTRY'] == 'Hungary'))]
df_actual = df_actual[~((df_actual['SUBBRAND'] == 'Lipton (L3)') & (df_actual['COUNTRY'] == 'Norway'))]


In [495]:
df_actual.reset_index(drop=True, inplace=True)

### **Datos cercanos a cero**

En el análisis anterior, se identificaron valores de `Amount` muy bajos, los cuales deberían revisarse antes de llegar a conclusiones definitivas. Estos valores cercanos a cero podrían indicar un posible declive de la marca en los últimos días.

Aunque también se observaron cantidades como 0.44, que no son lógicas en términos de ventas reales, ya que no se puede vender una fracción tan pequeña de un producto. Sin embargo, al no tratarse de valores negativos, interpretaremos estos datos como una señal del declive de la marca 7up en la República Checa. Por lo tanto, los valores aproximadamente iguales a cero se considerarán como cero, indicando que las ventas han cesado en ese mercado.

In [496]:
# Filtrar los datos de 7up en la República Checa
df_7up_Czech = df_actual[(df_actual["SUBBRAND"] == "7up (L3)") & (df_actual["COUNTRY"] == "Czech")]

# Convertir las fechas para mostrar solo el mes y el año
df_7up_Czech["Fecha"] = pd.to_datetime(df_7up_Czech["Fecha"]).dt.to_period("M").astype(str)

# Crear gráfico de líneas con plotly
fig = px.line(df_7up_Czech, x="Fecha", y="AMOUNT", title="7up en República Checa",
              labels={"Fecha": "Mes y Año", "AMOUNT": "Ventas"})

# Configurar rotación de etiquetas en el eje x
fig.update_xaxes(tickangle=45, tickfont=dict(size=10))

# Mostrar el gráfico
fig.show()

<ipython-input-496-8f5091bf9a49>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## **Distribución de los datos**

En este apartado, se analizará la distribución de las ventas de PepsiCo en tres dimensiones clave: por país, por mes y año, y por cada sub-marca. Este enfoque permite comprender mejor cómo varía la demanda de Pepsi Max en diferentes contextos geográficos, temporales y de producto.

### **Datos máximos y distribución de países y submarcas**

Ahora vamso a ver cuales son las máximas cantidades por países y por marcas, para ello vasmoa a utilizar un gráfico de bigotes para mostrar la distribución de los datos, identificar cuales son los datos máximos y estadísiticos básicos como los cartiles.

In [542]:
# Crear una figura de subplots con Plotly
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Distribución de Amount por Pais", "Distribución de Amount por Submarca"))

# Primer gráfico: Distribución de Amount por Country
fig_country = px.box(df_actual, x='COUNTRY', y='AMOUNT', title="Distribución de Amount por Pais")
for trace in fig_country.data:
    fig.add_trace(trace, row=1, col=1)

# Segundo gráfico: Distribución de Amount por Subbrand
fig_subbrand = px.box(df_actual, x='SUBBRAND', y='AMOUNT', title="Distribución de Amount por Submarca")
for trace in fig_subbrand.data:
    fig.add_trace(trace, row=1, col=2)

# Ajustar el diseño de la figura
fig.update_layout(
    title="Distribución de Amount por Pais y Submarca (datos completos)",
    xaxis=dict(tickangle=45),  # Rotar etiquetas en el primer gráfico
    xaxis2=dict(tickangle=45),  # Rotar etiquetas en el segundo gráfico
    width=1000,
    height=600,
    showlegend=False
)

# Mostrar la gráfica
fig.show()

En la gráfica de la columna de países, **Gran Bretaña** se destaca con una distribución de ventas que incluye valores extremadamente altos en comparación con los otros países. Observamos que la caja de distribución de Gran Bretaña tiene un rango intercuartil amplio, lo que indica variabilidad en las ventas medianas. Además, existen varios valores atípicos que superan los 10 millones, llegando hasta aproximadamente 14 millones, lo que resalta la importancia de Gran Bretaña como un mercado significativo con picos de ventas muy altos.

En la gráfica de la columna de marcas, **Pepsi Max (L3)** muestra una distribución de ventas considerablemente amplia. Su rango intercuartil es uno de los mayores entre todas las marcas, indicando una gran variabilidad en el volumen de ventas. La mediana se sitúa en una posición elevada en comparación con otras marcas, lo que sugiere que, en promedio, las ventas de Pepsi Max son consistentemente altas. Además, existen outliers en el rango superior, algunos alcanzando hasta 14 millones, lo que confirma que Pepsi Max experimenta picos de demanda en ciertos períodos, unicamnete en el país de Gran Bretaña.


**Pepsi y Gran bretaña**

Observamos que existen valores atípicos asociados al Reino Unido, especialmente relacionados con la marca Pepsi Max. Aunque es comprensible que haya diferencias debido a la mayor población del Reino Unido (68.35 millones) en comparación con la de España (48.37 millones), la magnitud de esta discrepancia es considerablemente excesiva. La variación supera los 10^7 mienstres que el resto oscilan en torno al valor de 10^6, lo cual no se ajusta a una diferencia poblacional plausible, sino que indica la presencia de datos fuera de escala. Estos registros serán considerados como *outliers* y eliminados temporalemnte para favorecer la visualización de la distribución de los datos, en marcas y países.


In [498]:
# Filtrar los datos de Pepsi Max en Great Britain
df_max_GB = df_actual[(df_actual["SUBBRAND"] == "Pepsi Max (L3)") & (df_actual["COUNTRY"] == "Great Britain")]

# Convertir las fechas para mostrar solo el mes y el año
df_max_GB["Fecha"] = pd.to_datetime(df_max_GB["Fecha"]).dt.to_period("M").astype(str)

# Crear la gráfica de líneas
fig = px.line(df_max_GB, x="Fecha", y="AMOUNT", title="Pepsi Max en Gran Bretaña", labels={"Fecha": "Mes y Año", "AMOUNT": "Ventas"})

# Ajustar el diseño de las etiquetas de fecha
fig.update_xaxes(tickangle=45, tickfont=dict(size=10))

# Mostrar la gráfica
fig.show()

<ipython-input-498-417c37306c05>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**Distribución de ventas por países paises**

Al analizar la distribución de las ventas por país y sub-marca sin incluir a Gran Bretaña, se confirma que este último tiene las ventas más altas. Sin embargo, observamos que países como Noruega tienen en general ventas bajas, con excepción de algunos casos específicos como las ventas de *Pepsi Max*, que se destacan. En este contexto, el máximo de ventas en Noruega, sin contar a *Pepsi Max*, apenas alcanza las 238,323 unidades. Un patrón similar se observa en Dinamarca, donde el máximo sin *Pepsi* llega a 394,678 ventas. En el caso de España, el volumen medio de ventas es el más bajo de todos los países, probablemente debido a una preferencia de los consumidores por la marca Coca-Cola.

Para el resto de los países, la distribución indica una alta concentración de ventas en rangos bajos (miles de unidades), mientras que los valores más dispersos se encuentran en los cuartiles superiores. Esto podría sugerir que las ventas en millones no son frecuentes en todos los meses, ya que dependen de múltiples factores como la estacionalidad, situación económica o celebraciones. La popularidad de ciertos productos, como los de la marca *Pepsi*, también influye en estas variaciones, reflejando cómo la marca tiene un impacto notable en los volúmenes de venta en algunos mercados.

In [499]:
df_no_british = df_actual[~(df_actual["COUNTRY"] == "Great Britain")]
df_no_pepsi = df_actual[~(df_actual["SUBBRAND"] == "Pepsi Max (L3)")]

In [543]:
# Crear una figura de subplots con Plotly
fig = sp.make_subplots(rows=1, cols=2, subplot_titles=("Distribución de Amount por Pais", "Distribución de Amount por Submarca"))

# Primer gráfico: Distribución de Amount por Country
fig_country = px.box(df_no_british, x='COUNTRY', y='AMOUNT', title="Distribución de Amount por Pais")
for trace in fig_country.data:
    fig.add_trace(trace, row=1, col=1)

# Segundo gráfico: Distribución de Amount por Subbrand
fig_subbrand = px.box(df_no_pepsi, x='SUBBRAND', y='AMOUNT', title="Distribución de Amount por Submarca")
for trace in fig_subbrand.data:
    fig.add_trace(trace, row=1, col=2)

# Puntos individuales solo para Pepsi
df_pepsi = df_no_british[df_no_british["SUBBRAND"].str.contains("Pepsi", case=False)]
fig.add_trace(
    go.Scatter(
        x=df_pepsi["COUNTRY"],
        y=df_pepsi["AMOUNT"],
        mode='markers',
        name='Datos Pepsi',
        marker=dict(color='red', size=6, opacity=0.6)
    ),
    row=1, col=1
)

# Ajustar el diseño de la figura
fig.update_layout(
    title="Distribución de Amount por Pais y Bubmarca",
    xaxis=dict(tickangle=45),  # Rotar etiquetas en el primer gráfico
    xaxis2=dict(tickangle=45),  # Rotar etiquetas en el segundo gráfico
    width=1000,
    height=600,
    showlegend=False
)

# Mostrar la gráfica
fig.show()

### **Distirbución de meses y años**

In [501]:
# Agrupar los datos por mes/año y marca, sumando la cantidad
df_grouped = df_actual.groupby(["Fecha", "SUBBRAND"])["AMOUNT"].sum().reset_index()

# Crear el histograma
fig = px.bar(df_grouped, x="Fecha", y="AMOUNT", color="SUBBRAND", title="Cantidad por Marca en cada Mes y Año",
             labels={"Fecha": "Fecha", "AMOUNT": "Cantidad", "SUBBRAND": "Marca"})

# Ajustar el diseño para una mejor visualización
fig.update_layout(
    xaxis=dict(tickangle=45),  # Rotar etiquetas para legibilidad
    barmode='stack'            # Apilar las barras para ver el total por mes/año
)

# Mostrar la gráfica
fig.show()

Observamos que Pepsi Max (L3) (en color naranja) es consistentemente la marca con la mayor cantidad de ventas en la mayoría de los meses, con una contribución significativa al total mensual. Pepsi Regular (L3) (en color azul claro) también mantiene una presencia notable, posicionándose generalmente como la segunda mayor contribuyente en cada mes. Las otras marcas, como 7up (L3), 7up Free (L3), Lipton (L3), y Mountain Dew (L3), muestran contribuciones menores en comparación, con cantidades de ventas más reducidas y menos variabilidad entre los meses.

Enero se destaca consistentemente como una época de ventas bajas, mostrando los valores más reducidos en comparación con otros meses del año. Esta tendencia podría asociarse con un período de menor consumo después de las festividades de fin de año, cuando la demanda disminuye.

Por otro lado, agosto se presenta como la época alta, con un aumento notable en las ventas totales. Este incremento es evidente en la altura de las barras en comparación con otros meses, reflejando un repunte en el consumo, probablemente debido a las vacaciones de verano y el clima cálido en muchas regiones, que tiende a estimular el consumo de bebidas. Este patrón de baja en enero y alza en agosto se repite en ambos años observados, sugiriendo una estacionalidad consistente en la demanda de estas marcas a lo largo del tiempo.


In [502]:
# Agrupar los datos por mes/año y sumar el AMOUNT de todas las marcas
df_total = df_actual.groupby("Fecha")["AMOUNT"].sum().reset_index()

# Convertir 'Fecha' a formato de fecha y ordenar los datos
df_total["Fecha"] = pd.to_datetime(df_total["Fecha"])
df_total = df_total.sort_values("Fecha")

# Calcular la media móvil de 3 meses para ver la estacionalidad general
df_total["Moving_Avg"] = df_total["AMOUNT"].rolling(window=3, min_periods=1).mean()

# Crear el gráfico de barras con la suma total por mes/año
fig = px.bar(df_total, x="Fecha", y="AMOUNT", title="Cantidad Total por Mes y Año con Estacionalidad",
             labels={"Fecha": "Fecha", "AMOUNT": "Ventas total"})

# Añadir la línea de estacionalidad general
fig.add_trace(go.Scatter(
    x=df_total["Fecha"],
    y=df_total["Moving_Avg"],
    mode='lines',
    name="Estacionalidad Total",
    line=dict(color="black", width=2)  # Ajusta el color y ancho de la línea si lo deseas
))

# Filtrar todos los puntos de agosto y febrero
august_points = df_total[df_total["Fecha"].dt.month == 8]
february_points = df_total[df_total["Fecha"].dt.month == 2]

# Añadir puntos máximos en agosto con una única entrada en la leyenda
fig.add_trace(go.Scatter(
    x=august_points["Fecha"],
    y=august_points["Moving_Avg"],
    mode='markers+text',
    name="Agosto",
    marker=dict(color="blue", size=10),
    text=["Máximo"] * len(august_points),
    textposition="top center"
))

# Añadir puntos mínimos en febrero con una única entrada en la leyenda
fig.add_trace(go.Scatter(
    x=february_points["Fecha"],
    y=february_points["Moving_Avg"],
    mode='markers+text',
    name="Febrero",
    marker=dict(color="green", size=10),
    text=["Mínimo"] * len(february_points),
    textposition="top center"
))

# Ajustar el diseño para una mejor visualización
fig.update_layout(
    xaxis=dict(tickangle=45),  # Rotar etiquetas para legibilidad
)

# Mostrar la gráfica
fig.show()

La gráfica muestra las ventas totales mensuales con una línea que representa la tendencia general a lo largo del tiempo. Esta línea de media móvil de 3 meses se usa para suavizar los datos y ayudar a identificar patrones estacionales.

Podemos observar que, en general, durante los meses de enero y febrero se produce un ligero declive en las ventas, mientras que en los meses de verano, especialmente entre junio y agosto, las ventas tienden a aumentar. Esto podría reflejar patrones de comportamiento de los consumidores o factores estacionales que afectan la demanda en esos periodos. La línea de media móvil permite ver estos patrones de manera más clara, sin dejarse influenciar por variaciones puntuales en los datos. Esta gráfica, sirve para complementar la información que observamso en la gráfica anterior a esta, donde apreciamso la distribución de las maras en cada mes, y que lso patrones estacionales se muestran en cada marca.

## **Tendencia y estacionalidad**

La **tendencia** refleja la dirección general de crecimiento o decrecimiento en una serie temporal a lo largo del tiempo, mientras que la **estacionalidad** muestra patrones cíclicos y recurrentes en intervalos regulares, influenciados por factores externos como el clima o eventos específicos.

### **País con menos ventas y Marca con más ventas**

- Pais con más ventas: Gran Bretaña
- Pais con menos ventas: Spain
- Marca con más ventas: Pepsi Max
- Marca con menos ventas: Mountain Dew

In [503]:
# Calcular la media de AMOUNT por COUNTRY y ordenar de mayor a menor
df_country_mean = df_actual.groupby('COUNTRY')['AMOUNT'].mean().reset_index()
df_country_mean.columns = ['COUNTRY', 'MEAN_AMOUNT_COUNTRY']
df_country_mean = df_country_mean.sort_values(by='MEAN_AMOUNT_COUNTRY', ascending=False)

# Calcular la media de AMOUNT por SUBBRAND y ordenar de mayor a menor
df_subbrand_mean = df_actual.groupby('SUBBRAND')['AMOUNT'].mean().reset_index()
df_subbrand_mean.columns = ['SUBBRAND', 'MEAN_AMOUNT_SUBBRAND']
df_subbrand_mean = df_subbrand_mean.sort_values(by='MEAN_AMOUNT_SUBBRAND', ascending=False)

# Crear el gráfico de barras para la media de AMOUNT por COUNTRY
fig_country = px.bar(df_country_mean, x='COUNTRY', y='MEAN_AMOUNT_COUNTRY', title='Media de Ventas - Country',
                     labels={'COUNTRY': 'Country', 'MEAN_AMOUNT_COUNTRY': 'Media de Ventas'})
fig_country.update_traces(text=df_country_mean['MEAN_AMOUNT_COUNTRY'].round(2).astype(str), textposition='outside')

# Crear el gráfico de barras para la media de AMOUNT por SUBBRAND
fig_subbrand = px.bar(df_subbrand_mean, x='SUBBRAND', y='MEAN_AMOUNT_SUBBRAND', title='Media de Ventas - Subbrand',
                      labels={'SUBBRAND': 'Subbrand', 'MEAN_AMOUNT_SUBBRAND': 'Media de Ventas'})
fig_subbrand.update_traces(text=df_subbrand_mean['MEAN_AMOUNT_SUBBRAND'].round(2).astype(str), textposition='outside')

# Crear un layout de subplots y agregar ambos gráficos
fig = make_subplots(rows=1, cols=2, subplot_titles=("Media de Ventas - Country", "Media de Ventas - Subbrand"))

# Añadir gráficos al layout
fig.add_traces(fig_country['data'], rows=1, cols=1)
fig.add_traces(fig_subbrand['data'], rows=1, cols=2)

# Ajustar el diseño y mostrar
fig.update_layout(title_text="Media de Ventas por Pais y Submarca", showlegend=False)
fig.show()


### **España: menor venta**

In [504]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from scipy.stats import linregress

# Filtrar datos para España y calcular las ventas por mes
df_spain = df_actual[df_actual['COUNTRY'] == 'Spain'].groupby('Fecha')['AMOUNT'].sum().reset_index()

# Cálculo de la tendencia
x = np.arange(len(df_spain))
y = df_spain['AMOUNT'].values
pendiente, intercepto, _, _, _ = linregress(x, y)
tendencia = pendiente * x + intercepto

# Creación del gráfico original
fig = px.line(df_spain, x='Fecha', y='AMOUNT', title='Tendencia y estacionalidad para España (mínima ventas)',
              labels={'Fecha': 'Fecha', 'AMOUNT': 'Cantidad de ventas'})
fig.update_traces(marker=dict(size=10))  # Ajuste del tamaño de los marcadores

# Añadir la línea de tendencia como línea discontinua
fig.add_trace(go.Scatter(
    x=df_spain['Fecha'],
    y=tendencia,
    mode='lines',
    name='Tendencia',
    line=dict(color='red')  # Línea roja discontinua
))

# Resaltar los puntos de agosto y febrero
df_spain['Fecha'] = pd.to_datetime(df_spain['Fecha'])  # Asegurarse de que 'Fecha' esté en formato de fecha

# Filtrar para agosto y febrero
agosto = df_spain[df_spain['Fecha'].dt.month == 8]
febrero = df_spain[df_spain['Fecha'].dt.month == 2]

# Añadir puntos grandes en agosto
fig.add_trace(go.Scatter(
    x=agosto['Fecha'],
    y=agosto['AMOUNT'],
    mode='markers',
    name='Maximo-Agosto',
    marker=dict(color='blue', size=12, symbol='circle')  # Punto grande en azul para agosto
))

# Añadir puntos grandes en febrero
fig.add_trace(go.Scatter(
    x=febrero['Fecha'],
    y=febrero['AMOUNT'],
    mode='markers',
    name='Mínimo-Febrero',
    marker=dict(color='green', size=12, symbol='circle')  # Punto grande en verde para febrero
))

# Mostrar el gráfico
fig.show()


Como se observa

**Análisis de la Tendencia**

La línea de tendencia roja discontinua tiene una pendiente ascendente, lo que sugiere un crecimiento gradual de las ventas de la marca PepsiCo, que comprende las 6 submarcas anteriores en Europa, en España a lo largo del tiempo. Al formar parte de la misma marca, se analiza como la suma de las ventas.
Aunque el crecimiento no es pronunciado, se observa que, en promedio, las ventas han estado aumentando. Esta tendencia ascendente implica que la demanda de esta marca podría estar ganando popularidad en el mercado español o que ciertos factores están favoreciendo el incremento de las ventas, como campañas de marketing, cambios en las preferencias de los consumidores o eventos estacionales recurrentes que favorecen las ventas.

**Análisis de la Estacionalidad**

Observando la línea azul de las ventas mensuales, se puede identificar un comportamiento estacional claro, con picos altos y bajos recurrentes en determinados períodos del año. Este patrón sugiere que las ventas están influenciadas por factores estacionales específicos. Algunos puntos clave incluyen:

- Picos altos en verano (Julio y Agosto): En ambos años analizados, las ventas alcanzan sus niveles más altos en julio y agosto, con el punto máximo en julio de 2023, donde superan las 550,000 unidades. Este aumento coincide con los meses más cálidos, lo que probablemente favorece el consumo de bebidas, impulsando así la demanda.

- Picos bajos en invierno (Enero y Febrero): En contraste, los niveles de ventas más bajos se observan consistentemente en enero y febrero de cada año. En el período de diciembre 2023 a enero 2024, las ventas caen por debajo de las 400,000 unidades, probablemente debido a una reducción en el consumo después de las festividades, cuando los consumidores tienden a ajustar sus gastos.

- Patrón de recuperación en verano: Al acercarse el verano de 2024, las ventas comienzan a aumentar nuevamente, reflejando un patrón estacional similar al del año anterior. Esto refuerza la influencia de factores climáticos y vacacionales en la demanda, con un volumen de ventas significativamente mayor en los meses cálidos.

### **Pepsi Max (L3): máxima venta**

In [505]:
# Filtrar datos para Pepsi Max y calcular las ventas por fecha
df_pepsi = df_actual[df_actual['SUBBRAND'] == 'Pepsi Max (L3)'].groupby('Fecha')['AMOUNT'].sum().reset_index()

# Cálculo de la tendencia
x = np.arange(len(df_pepsi))
y = df_pepsi['AMOUNT'].values
pendiente, intercepto, _, _, _ = linregress(x, y)
tendencia = pendiente * x + intercepto

# Creación del gráfico original
fig = px.line(df_pepsi, x='Fecha', y='AMOUNT', title='Tendencia y estacionalidad para Pepsi Max (máxima venta)',
              labels={'Fecha': 'Fecha', 'AMOUNT': 'Cantidad de ventas'})
fig.update_traces(marker=dict(size=10))  # Ajuste del tamaño de los marcadores

# Añadir la línea de tendencia como línea discontinua
fig.add_trace(go.Scatter(
    x=df_pepsi['Fecha'],
    y=tendencia,
    mode='lines',
    name='Tendencia',
    line=dict(color='red')  # Línea roja discontinua
))

# Aplicar el patrón de mínimos y máximos
min_indices = list(range(0, len(df_pepsi), 3))  # Mínimos cada 3 meses

# Cálculo de índices para máximos con el nuevo patrón: 2, 1, 1, 2, 2, ...
max_indices = []
max_pattern = [2, 1, 1, 2]  # Patrón de alternancia
pattern_length = len(max_pattern)

for i, min_idx in enumerate(min_indices):
    max_idx = min_idx + max_pattern[i % pattern_length]  # Aplicar el patrón de alternancia

    if max_idx < len(df_pepsi):  # Asegurarse de que el índice está en el rango de datos
        max_indices.append(max_idx)

# Añadir puntos para los mínimos
fig.add_trace(go.Scatter(
    x=df_pepsi['Fecha'].iloc[min_indices],
    y=df_pepsi['AMOUNT'].iloc[min_indices],
    mode='markers',
    name='Mínimos',
    marker=dict(color='blue', size=12, symbol='circle')  # Punto grande en azul para mínimos
))

# Añadir puntos para los máximos
fig.add_trace(go.Scatter(
    x=df_pepsi['Fecha'].iloc[max_indices],
    y=df_pepsi['AMOUNT'].iloc[max_indices],
    mode='markers',
    name='Máximos',
    marker=dict(color='green', size=12, symbol='circle')  # Punto grande en verde para máximos
))

# Mostrar el gráfico
fig.show()


La gráfica muestra la tendencia y estacionalidad de las ventas de Pepsi Max en España desde el 1 de enero hasta agosto del año siguiente. La serie temporal incluye tanto una línea de tendencia, calculada con regresión lineal, como puntos específicos marcados como mínimos y máximos, siguiendo un patrón preestablecido.

**Tendencia ascendente:**

Se aprecia claramente una tendencia positiva en el número de ventas. La línea de tendencia roja discontinua muestra una pendiente ascendente, lo que sugiere un crecimiento consistente en la demanda de Pepsi Max a lo largo del tiempo. Este patrón indica una expansión en el mercado o una popularidad creciente de la marca, posiblemente impulsada por factores como el posicionamiento en el mercado y estrategias de marketing.

**Análisis de la Estacionalidad**

- **Mínimos (trimestrales):** Se destacan cada 3 meses, lo que podría indicar una caída periódica en las ventas de Pepsi Max en estos intervalos. La aparición regular de estos mínimos sugiere que hay ciertos momentos, cada trimestre, donde la demanda disminuye, posiblemente reflejando una tendencia estacional en el consumo.

- **Máximos:** Se identifican en un patrón alternante: 2 meses después de un mínimo, luego 1 mes después, seguido nuevamente por 1 mes y luego 2 meses después, y así sucesivamente. Este patrón sugiere que después de cada descenso en las ventas, hay una recuperación que sigue esta secuencia. Esto podría estar relacionado con promociones, eventos o cambios estacionales que impulsan temporalmente las ventas.

## **Análisis de datos de predicción**

En primer lugar, ajustaremos el formato de los valores en la columna de predicciones, transformando el formato AI_P(mes)F en un valor numérico de mes. Esto permitirá una mejor interpretación temporal de las predicciones y su comparación con los datos reales. Posteriormente, realizaremos un análisis inicial mediante el cálculo de estadísticos descriptivos para los campos numéricos (año, mes y volumen de ventas) en ambos conjuntos. Esta revisión es fundamental para evaluar la coherencia y cobertura temporal de los datos, y nos permitirá verificar si los periodos de predicción se alinean con los años de datos reales disponibles. Finalmente, determinaremos el número de países y sub-marcas representados en los datos, lo cual aportará claridad sobre la complejidad y dimensión del problema de análisis y modelado.


Finalmente, calculamos los estadísticos básicos para las columnas numéricas en los datos reales y en las predicciones para observar la distribución de cada subconjunto.

In [506]:
# Primero hay que cambair a dos dataset
df_forecast['FORECAST_YEAR'] = df_forecast['FORECAST_YEAR'].astype(int)

<ipython-input-506-ca7a3dae5826>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Los datos se ajustarán como si fueran 1 día después para facilitar futuras representaciones. Por tanto, el dato AI_P02F del 2023, que sería 31 de Enero del 2023, se considerará 1 de Febrero del 2023, que para la finalidad del ejericio no cambia la lógica.

In [507]:
# cambiar a 12 es 1 y el resto van a su mes
import re

def extract_number(text):
    match = re.search(r'\d+', text) # Mira lo que sea texto
    if match:
        return int(match.group(0)) #Coge lo que no sea letra
    else:
        return 1  # Si no hya nada es proque es el mes enero

df_forecast['FORECAST'] = df_forecast['FORECAST'].apply(extract_number)

<ipython-input-507-3074409a2ed1>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [508]:
df_forecast[['YEAR', 'MONTH', 'FORECAST', 'FORECAST_YEAR', 'AMOUNT']].describe()

,YEAR,MONTH,FORECAST,FORECAST_YEAR,AMOUNT
count,17766.000000,17766.000000,17766.000000,17766.0,1.776600e+04
mean,2023.732298,6.514860,6.802432,2023.0,9.837240e+05
std,0.590992,3.472165,3.430173,0.0,1.915109e+06
min,2023.000000,1.000000,1.000000,2023.0,0.000000e+00
25%,2023.000000,3.000000,4.000000,2023.0,8.917303e+04
50%,2024.000000,7.000000,7.000000,2023.0,3.239128e+05
75%,2024.000000,10.000000,10.000000,2023.0,1.100389e+06
max,2025.000000,12.000000,12.000000,2023.0,1.373810e+07


1. **Año de Predicción:**

Los años abarcan desde 2023 hasta 2025, con una media cercana a 2024. Esto indica que el horizonte de predicción cubre aproximadamente tres años, con una mayoría concentrada en 2023 y 2024.

2. **Mes de Predicción:**

Los valores en MONTH va de 1 a 12, cubriendo todos los meses. Esto garantiza que hay datos de predicción para todos los meses del año, permitiendo una evaluación de estacionalidad.

3. **Volumen de Ventas (AMOUNT):**

La media de ventas predichas es 983,724, pero existe una alta variabilidad (desviación estándar de 1,915,109) con valores que oscilan desde 0 hasta 13,738,100. Esto indica una amplia distribución en los volúmenes de ventas esperados, posiblemente debido a variaciones entre países, productos, o estacionalidad en el tiempo. El 0 implica que no se predijo ninguna venta (observar más adelante).

4. **Consistencia Temporal:**

El valor constante en FORECAST_YEAR (2023) sugiere que todas las predicciones se han realizado con base en los datos observados de ese año, lo cual puede ser relevante al evaluar la precisión de estos pronósticos.

**Horizonte de predicción**

El horizonte de predicción es el periodo de tiempo hacia el futuro para el cual un modelo o método de pronóstico realiza sus estimaciones. En otras palabras, es la extensión temporal que cubre la predicción, definida en unidades de tiempo como días, meses, trimestres o años. En nuestro caso se basa en meses.

Para ello, hay que tener en cuanta que hablamos de una sub-marca, en un lugar, y con la predicción qu ese haya hecho en un mes en particular.

El horizonte de predicción conincide para todas las marcas, paises y mes en el que se hizo la predicción.

In [509]:
# Sacar el horizonte de predicción

def horizonte(df):
  i = 0
  for year in range(2023, 2026, 1):
    df_year = df[df['YEAR'] == year]
    for month in range(1, 13, 1):
      if month in df_year['MONTH'].values:
        i = i + 1
  return i

In [510]:
print(f'El horizonte de predicción es de {horizonte(df_forecast[(df_forecast["SUBBRAND"] == "Lipton (L3)") & (df_forecast["COUNTRY"] == "Spain") & (df_forecast["FORECAST"] == 1)])} meses : Lipton')


El horizonte de predicción es de 18 meses : Lipton


In [511]:
print(f'El horizonte de predicción es de {horizonte(df_forecast[(df_forecast["SUBBRAND"] == "7up (L3)") & (df_forecast["COUNTRY"] == "Spain") & (df_forecast["FORECAST"] == 1) & (df_forecast["FORECAST"] == 1)])} meses: 7up')

El horizonte de predicción es de 18 meses: 7up


**¿Existen varias predicciones de una misma marca en un mismo pais y para un solo mes?**

A continuación, realizaremos una comparación detallada de las predicciones para el mes de enero de 2024 de la compañía Lipton en España, con el objetivo de evaluar la precisión y evolución de estas proyecciones. Para ello:

- **Predicciones para enero de 2024 realizadas a lo largo de 2023:** Dibujaremos las predicciones realizadas para enero de 2024 en cada uno de los meses de 2023, lo que permitirá observar cómo se han ajustado estas estimaciones a medida que se disponía de más datos reales durante el año. Este análisis mostrará si las proyecciones han sido consistentes o si han variado significativamente con el tiempo.

- **Datos reales de 2023:** Incluir los datos reales de ventas mensuales de 2023 permitirá contextualizar la precisión de las predicciones. Al comparar estos valores con las proyecciones para enero de 2024, podremos ver si el modelo ha incorporado de manera efectiva los patrones y tendencias observadas durante el año anterior.

- **Valor real de enero de 2024:** Finalmente, compararemos estas predicciones con el valor real de ventas en enero de 2024. Esto nos permitirá evaluar si los ajustes realizados en las proyecciones a lo largo de 2023 han llevado a una predicción acertada o si existe una discrepancia significativa.

In [512]:
# df_forecast[(df_forecast["SUBBRAND"] == "Lipton (L3)") & (df_forecast["COUNTRY"] == "Spain") & (df_forecast["MONTH"] == 1) & (df_forecast["YEAR"] == 2024)].sort_values(by="FORECAST")


In [513]:
# df_actual[(df_actual["SUBBRAND"] == "Lipton (L3)") & (df_actual["COUNTRY"] == "Spain") & (df_actual["YEAR"] == 2023)].sort_values(by="MONTH")

In [514]:
# Filtrar prediccion hecha para enero, relaizada en todos los meses del 2023
predic_enero = df_forecast[
    (df_forecast["COUNTRY"] == "Spain") &
    (df_forecast["SUBBRAND"] == "Lipton (L3)") &
    (df_forecast["YEAR"] == 2024) &
    (df_forecast["MONTH"] == 1)
].sort_values(by="FORECAST")

# Filtrar prediccion hecha para enero, relaizada en todos los meses del 2023
real_2023 = df_actual[
    (df_actual["COUNTRY"] == "Spain") &
    (df_actual["SUBBRAND"] == "Lipton (L3)") &
    (df_actual["YEAR"] == 2023)
].sort_values(by="MONTH")

# Filtrar los índices que cumplen con las condiciones especificadas
indices_valor1_24 = df_actual.index[
    (df_actual["COUNTRY"] == "Spain") &
    (df_actual["SUBBRAND"] == "Lipton (L3)") &
    (df_actual["YEAR"] == 2024) &
    (df_actual["MONTH"] == 1)
]

value = int(df_actual.loc[indices_valor1_24, 'AMOUNT'])

<ipython-input-514-f650690ac5df>:24: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead



In [515]:
# Crear la figura
fig = go.Figure()

# Agregar la línea de predicciones para 1/24
fig.add_trace(go.Scatter(
    x=predic_enero["FORECAST"],
    y=predic_enero["AMOUNT"],
    mode='lines',
    name='Predicciones para 1/24',
    line=dict(color='blue', width=2)
))

# Agregar la línea de datos reales para 2023
fig.add_trace(go.Scatter(
    x=real_2023["MONTH"],
    y=real_2023["AMOUNT"],
    mode='lines',
    name='Real del 2023',
    line=dict(color='green', width=2)
))

# Agregar la línea horizontal para el valor real de 1/24
fig.add_hline(
    y=value,
    line_color="red",
    line_dash="dot",
    line_width=2,
    name="Valor real de 1/24"
)

fig.add_trace(go.Scatter(
    x=[None], y=[None],
    mode='lines',
    line=dict(color='red', dash='dot', width=2),
    name="Valor real de 1/24"
))

# Configurar el diseño de la leyenda y etiquetas
fig.update_layout(
    title="Predicción del 1 de Enero en España para Lipton: Precisión",
    xaxis_title="Meses",
    yaxis_title="Amount",
    legend_title="Series",
    legend=dict(
        title_font_size=12,
        font_size=10,
        bordercolor="Black",
        borderwidth=1,
        bgcolor="LightGray"
    )
)

# Mostrar la gráfica
fig.show()


Esta gráfica nos ayuda a comprender cómo funcionan las predicciones y cómo se crean. En cada mes, se muestran los datos predichos para el 1 de enero de 2024 (línea azul), basándose en la información disponible hasta ese momento. La línea verde representa los datos reales del 2023. Observamos que cuando hay un aumento en los datos reales, se refleja también en las predicciones. Además, en general, los valores predichos para el 1 de enero suelen ser más optimistas que los datos reales del mes donde se ha realizado la predicción. Además, podemos observar que en general los datos que mejor se ajustan son los de ese mimso mes un año antes (por la estacionalidad de los datos) o los datos realizados un mes antes de los datos.

**Vamos a crear la columna Fecha**

Para poder visulizar correctamente las series temporales.

In [516]:
df_forecast['Fecha'] = pd.to_datetime(df_forecast[['YEAR', 'MONTH']].assign(DAY=1))
df_forecast = df_forecast.sort_values(by="Fecha")

<ipython-input-516-1fb03de728ea>:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## **Errores de predicción**

- Sales Amount over Time for {subbrand} (Spain)
- Porcentaje de la desviación estandar del error de predicción por marca
- Desviación estandar de la predicción en función de la diferencia temporal

In [517]:
df_spain_act = df_actual[df_actual['COUNTRY'] == 'Spain']
df_spain_for = df_forecast[df_forecast['COUNTRY'] == 'Spain']

**Identificación de varias predicciones hechas en un mismo mes para un mismo dato real:**

Por ejemplo, para Pepsi Regular en España, es posible que haya tres predicciones diferentes hechas en abril de 2023 para el dato real de enero de 2024. Este fenómeno introduce redundancia en las predicciones y dificulta el análisis de precisión.

**Solución:**

Para abordar este inconveniente y permitir un análisis preciso de la precisión del modelo, vamos a calcular la media de las predicciones realizadas en el mismo mes para cada dato real. Al agrupar y promediar las predicciones repetidas, podremos consolidar los datos, eliminando el ruido causado por múltiples predicciones y obteniendo una estimación más representativa de la predicción para cada dato real.

In [518]:
df_spain_for = df_spain_for.groupby(['SUBBRAND', 'FORECAST', 'Fecha'])['AMOUNT'].mean().reset_index()

### **Pasos previos**

**Diferencia de Fecha**

In [519]:
# Asignar siempre 2023 como el año y convertir 'FORECAST' a fecha con el primer día del mes
df_spain_for['FechaReal'] = pd.to_datetime(df_spain_for.assign(year=2023, day=1)[['year', 'FORECAST', 'day']].rename(columns={'FORECAST': 'month'}))

# Assuming df_spain_for has columns 'FechaReal' and 'Fecha'
df_spain_for['Diferencia_Fechas'] = (df_spain_for['Fecha'] - df_spain_for['FechaReal']).dt.days / 30.44
df_spain_for['Diferencia_Fechas'] = df_spain_for['Diferencia_Fechas'].round()  # Redondear a meses completos

**Número de submarcas en España**

El número de submarcas es 5: Pepsi Regular (L3), Pepsi Max (L3), 7up Free (L3), 7up (L3) y Lipton (L3). Donde no aparece ningún registro de Mountain Dew (L3).

In [520]:
# Agrupar por marca y contar la cantidad de registros
brand_counts = df_spain_act.groupby('SUBBRAND')['SUBBRAND'].count()

# Crear el gráfico de torta
fig = px.pie(
    brand_counts,
    values=brand_counts.values,
    names=brand_counts.index,
    title='Cantidad de registros por marca en España'
)

fig.show()

In [521]:
# Merge the two dataframes on 'Fecha', 'SUBBRAND', and 'COUNTRY' to find corresponding values
merged_df = pd.merge(df_spain_act, df_spain_for[['Fecha', 'SUBBRAND', 'AMOUNT', 'FechaReal', 'Diferencia_Fechas']],
                     on=['Fecha', 'SUBBRAND'], how='inner', suffixes=('_act', '_for'))

# Calcular la diferencia entre las columnas 'AMOUNT' de ambos dataframes
merged_df['Diferencia_amount'] = merged_df['AMOUNT_act'] - merged_df['AMOUNT_for']

In [522]:
merged_df.describe()

,YEAR,MONTH,AMOUNT_act,Fecha,AMOUNT_for,FechaReal,Diferencia_Fechas,Diferencia_amount
count,855.000000,855.000000,855.000000,855,855.000000,855,855.000000,855.000000
mean,2023.543860,6.192982,82715.986029,2023-12-22 23:43:09.473684224,80401.792988,2023-05-24 19:13:41.052631552,6.964912,2314.193041
min,2023.000000,1.000000,6855.564870,2023-01-01 00:00:00,4839.238140,2023-01-01 00:00:00,0.000000,-64956.266067
25%,2023.000000,4.000000,63363.062934,2023-09-01 00:00:00,52874.923789,2023-03-01 00:00:00,3.000000,-5816.193854
50%,2024.000000,6.000000,92671.386943,2024-01-01 00:00:00,81783.595995,2023-05-01 00:00:00,7.000000,227.668007
75%,2024.000000,8.000000,111719.093644,2024-05-01 00:00:00,110645.593783,2023-08-01 00:00:00,10.000000,10884.747531
max,2024.000000,12.000000,186616.622952,2024-08-01 00:00:00,238326.723190,2023-12-01 00:00:00,17.000000,68485.220184
std,0.498364,3.243288,44334.137816,NaN,46296.948055,NaN,4.628967,16821.692210


#### **El análisis de las fechas:**

- **Horizonte de Predicción:** La columna Diferencia_Fechas muestra que las predicciones se realizan en un horizonte de 18 meses, de 0 a 17 meses. Esto significa que las predicciones pueden proyectarse hasta 18 meses en el futuro respecto a la fecha de predicción inicial. Así certificamso los datos, que estudiamos anteriormente.

- **Predicciones en 2023 y Ausencia de Comparaciones Directas para 2024:** Todas las predicciones fueron realizadas en 2023. Por lo tanto, cuando analizamos datos de 2024, no existen predicciones para el mismo mes de 2024 con una Diferencia_Fechas de 0. Esto implica que no es posible comparar el rendimiento de una predicción hecha para el mes actual (0 meses) con una predicción hecha para 17 meses en el futuro, ya que no hay datos correspondientes al mismo mes en 2024. Cualquier comparación entre predicciones a 0 y 17 meses es inadecuada porque se trata de periodos de tiempo distintos, con contextos y variables diferentes.

- **Diferencia_Fechas** representa el número de meses entre la fecha real y la fecha de predicción, con una media de 6.65 meses. Esto sugiere que, en promedio, las predicciones se hacen para un horizonte de alrededor de medio año.
La desviación estándar de 4.43 meses indica que existe una variabilidad considerable en los horizontes de predicción, con algunas predicciones a corto plazo y otras a largo plazo, lo que podría influir en la precisión de las mismas.
- **Análisis del Error:** Dado que no podemos comparar directamente las predicciones hechas a diferentes horizontes de tiempo (por ejemplo, 0 meses vs. 17 meses), el análisis se centrará en calcular y comparar el error estándar y la media del error en función de Diferencia_Fechas. Esto permitirá evaluar cómo varía la precisión de las predicciones según el horizonte temporal, proporcionando una visión más completa de los errores en los distintos horizontes de predicción.

#### **El análisis de ventas:**

- **AMOUNT_act** (cantidad real) tiene una media de 96,547.79, mientras que **AMOUNT_for** (cantidad predicha) tiene una media ligeramente superior, de 97,863.24. Esto indica que, en promedio, las predicciones tienden a ser un poco más optimistas que los valores reales.
Los valores mínimos para ambas columnas también destacan diferencias: la venta real más baja es de 6,855.56 mientras que la predicción más baja es de 4,839.24, lo que indica que incluso en escenarios de baja demanda, las predicciones subestimaron las ventas.
La desviación estándar de AMOUNT_act es de 40,746.28 y para AMOUNT_for es de 48,332.33, lo que sugiere una mayor variabilidad en las predicciones en comparación con los datos reales.

- **Diferencia_amount** muestra la diferencia entre las ventas reales y las predichas. La media de -1,315.45 indica una ligera sobreestimación en las predicciones en general.
Los valores de Diferencia_amount oscilan entre un mínimo de -165,740.15 y un máximo de 68,485.22, lo que indica que las predicciones a veces sobreestiman y otras subestiman de manera significativa. La desviación estándar de 23,905.86 refleja una alta variabilidad en la precisión de las predicciones, destacando posibles patrones estacionales o de comportamiento en las ventas que no han sido capturados con precisión por el modelo de predicción.

### **Gráficas de precisión**

In [523]:
def graficar_serie_temporal(merged_df, marca, lineas):
  # Filtrar datos para 7up en España
  df_7up_spain = merged_df[(merged_df['SUBBRAND'] == marca)]

  # Crear la figura de Plotly
  fig = go.Figure()

  # Agregar las predicciones para cada 'Diferencia_Fechas'
  for diferencia in df_7up_spain['Diferencia_Fechas'].unique():
      visible = True if diferencia in lineas else 'legendonly'  # Mostrar solo 0, 12, 17 al inicio
      fig.add_trace(go.Scatter(
          x=df_7up_spain[df_7up_spain['Diferencia_Fechas'] == diferencia]['Fecha'],
          y=df_7up_spain[df_7up_spain['Diferencia_Fechas'] == diferencia]['AMOUNT_for'],
          mode='lines',
          name=f'Predicción: Diferencia - {diferencia} meses)',
          line=dict(width=2),
          visible=visible
      ))

  # Añadir la línea real como una línea roja más gruesa
  fig.add_trace(go.Scatter(
      x=df_7up_spain['Fecha'],
      y=df_7up_spain['AMOUNT_act'],
      mode='lines',
      name='Valor Real',
      line=dict(color='red', width=3),
      visible=True
  ))

  # Configuración del título y etiquetas
  fig.update_layout(
      title=f'Serie temporal {marca}: Valor Real -vs- Predicciones<br><sup>Selecciona las líneas en la leyenda para mostrar u ocultar predicciones según Diferencia_Fechas</sup>',
      xaxis_title='Fecha',
      yaxis_title='Ventas'
  )

  # Mostrar la gráfica
  fig.show()


Mostrar las predicciones para años como 2025 o más adelante no sería útil en este caso, ya que no contamos con datos reales para esos periodos y, por lo tanto, no podemos evaluar la precisión de las predicciones. Las predicciones sin datos reales equivalentes no aportan valor al análisis comparativo.

La gráfica anterior, es un ejmplo para mostrar como son las predicciones en comparación con los datos reales. Por lo que solo se muestra un ejemplo, porque no ofrece información real sobre lo bueno que son las predicciones.

In [524]:
def graficar_desviacion_normalizada_producto(merged_df, producto):
    # Filtrar datos para el producto y país especificados
    df_producto_pais = merged_df[(merged_df['SUBBRAND'] == producto)]

    # Agrupar por 'Diferencia_Fechas' y calcular la desviación estándar y la media de 'Diferencia_amount'
    std_by_horizon = df_producto_pais.groupby('Diferencia_Fechas')['Diferencia_amount'].agg(sum_abs=lambda x: x.abs().sum(), count='count', min = 'min', max = 'max')

    # Normalizar la desviación estándar dividiéndola entre la media
    std_by_horizon['normalized_std'] = (std_by_horizon['sum_abs'] / std_by_horizon['count'])

    # Crear el gráfico de barras
    fig = px.bar(
        std_by_horizon,
        x=std_by_horizon.index,
        y='normalized_std',
        labels={'Diferencia_Fechas': 'Horizonte de Predicción (Meses)', 'normalized_std': 'Media de Error Absoluto'},
        title=f'Media de Error Absoluto para {producto} por Horizonte de Predicción'
    )

    # Mostrar el gráfico
    fig.show()

El error de precisión por cada marca se calcula sumando los errores absolutos en `Diferencia_amount` para cada marca y horizonte de predicción, y luego dividiendo esta suma por el número de predicciones realizadas en ese horizonte para dicha marca. Como hemos visto anteriormente, solo se analizan las predicciones que pueden compararse con datos reales, por lo que las predicciones a largo plazo suelen tener muy pocos registros disponibles, por lo que pueden tener errores menos representativos.

Este método permite evaluar la precisión de las predicciones en distintos horizontes, ponderando el error en función del número de predicciones y resaltando la fiabilidad de las predicciones a corto plazo.

#### **Precisión por cada marca y horizonte**

In [525]:
graficar_desviacion_normalizada_producto(merged_df, producto='7up (L3)')


In [526]:
graficar_serie_temporal(merged_df, '7up (L3)', [3, 12, 17])

**Marca 7up**

Para los datos de 7up en España, las predicciones más precisas corresponden a los horizontes de 17, 9 y 3 meses, mientras que el de 12 meses muestra el peor ajuste. En la gráfica, la línea morada (predicción con 3 meses de diferencia) captura algunos picos de manera efectiva, aunque la predicción de 17 meses también muestra buen ajuste, sin embargo se basa en solo tres registros, lo cual limita su representatividad. En la práctica, los horizontes deberían ajustarse sumando un mes, debido al cambio del 31 de diciembre al 1 de enero en los datos.

In [527]:
graficar_desviacion_normalizada_producto(merged_df, producto='Lipton (L3)')

In [528]:
graficar_serie_temporal(merged_df, 'Lipton (L3)', [0, 1, 2, 16, 17])

**Lipton**

Para Lipton, las predicciones más precisas se obtienen con un horizonte de 1 mes y de 16 meses, mientras que el peor ajuste ocurre con 4 meses de diferencia. La serie presenta estacionalidad, con picos en agosto y mínimos en febrero, lo que sugiere una distribución de errores similar a una normal. Esto implica que un horizonte de 6 meses podría reflejar datos opuestos, convirtiendo un máximo en un mínimo y viceversa.

In [529]:
graficar_desviacion_normalizada_producto(merged_df, producto='7up Free (L3)')

In [530]:
graficar_serie_temporal(merged_df, '7up Free (L3)', [2, 4, 14, 16])

**7up Free**

Para 7up Free, los horizontes más precisos son de 2 y 4 meses, seguidos de 7 y 14 meses. Los peores resultados se observan en los horizontes más largos, probablemente debido a una mayor variabilidad en los datos. Al igual que en Lipton, existe una marcada estacionalidad; sin embargo, en este caso, las predicciones son generalmente precisas e forma similar simpre.

In [531]:
graficar_desviacion_normalizada_producto(merged_df, producto='Pepsi Max (L3)')

In [532]:
graficar_serie_temporal(merged_df, 'Pepsi Max (L3)', [0, 1, 12, 14])

 **Pepsi Max**

 En Pepsi Max ocurre algo similar: a mayor distancia en meses, menor precisión en las predicciones. Los horizontes más precisos son con 0, 1 y 14 meses de diferencia. Debido a la estacionalidad, los horizontes de 5 y 17 meses resultan en peores predicciones, ya que representan periodos opuestos en el ciclo estacional.

In [533]:
graficar_desviacion_normalizada_producto(merged_df, producto='Pepsi Regular (L3)')

In [534]:
graficar_serie_temporal(merged_df, 'Pepsi Regular (L3)', [0, 2, 9, 12])

**Pepsi Regular**

Con Pepsi Regular, las predicciones mejoran con menores diferencias en meses, destacándose la periodicidad de los datos. Observamos que las situaciones en febrero de distintos años son similares, lo que explica por qué un horizonte de 11-12 meses también ofrece buenos resultados al alinearse con el ciclo anual.

#### **Precisión global para España**

Como los resultados en genral han sido bastante precisos para instancias de tiempo corta, se toma la decisión de evaluar el horizonte 0 para ver cual es la submarca que mejor ha sido prevista con un mes de distancia en término reales.

In [535]:
df_zero_distance = merged_df[merged_df['Diferencia_Fechas'] == 0]


In [536]:
df_spain_act_filtered = df_spain_act[df_spain_act['Fecha'] <= pd.to_datetime('2023-12-01')]

In [537]:
# Calculate mean and standard deviation of AMOUNT for each SUBBRAND in df_spain_act
brand_stats = df_spain_act_filtered.groupby('SUBBRAND')['AMOUNT'].agg(['mean', 'std'])


In [538]:
# Agrupar por 'Diferencia_Fechas' y calcular la desviación estándar y la media de 'Diferencia_amount'
std_by_horizon = df_zero_distance.groupby('SUBBRAND')['Diferencia_amount'].agg(sum_abs=lambda x: x.abs().sum(), count='count', min = 'min', max = 'max')

# Normalizar la desviación estándar dividiéndola entre la media
std_by_horizon['normalized'] = (std_by_horizon['sum_abs'] / std_by_horizon['count'])

**Error Absoluto Promedio** (Mean Absolute Error, MAE): Este cálculo es similar al MAE, que se utiliza para evaluar la precisión del modelo en valores absolutos, sin considerar el signo del error.

**Interpretación:** Esta métrica indica el promedio de la magnitud del error de predicción para cada horizonte, sin importar si las predicciones son mayores o menores que los valores reales. Es una medida de precisión simple que ayuda a comprender la magnitud del error de manera general.

In [539]:
std_by_horizon = std_by_horizon.merge(brand_stats, on='SUBBRAND', how='left')

# Calcular el error normalizado en valor absoluto
std_by_horizon['normalized_std'] = ((std_by_horizon['mean'] - std_by_horizon['normalized']) / std_by_horizon['std']).abs()

El **error normalizado** calcula la magnitud del error de predicción en función de la variabilidad de los datos. Se obtiene al restar el error absoluto promedio del valor medio y luego dividir esa diferencia por la desviación estándar. Finalmente, se toma el valor absoluto para reflejar la magnitud del error sin importar su dirección.

Esta métrica permite evaluar la precisión del modelo de forma relativa, comparando el error con la dispersión de los datos: valores más bajos indican que el error es pequeño en relación con la variabilidad del conjunto de datos, lo cual sugiere una mejor precisión.

In [540]:
# Assuming your DataFrame is named 'merged_df'
std_by_horizon = std_by_horizon.reset_index()  # Resets the index if it's not already a column


In [541]:
# Suponiendo que 'std_by_horizon' es tu DataFrame
# Ordenar el DataFrame de mayor a menor por 'normalized_std'
std_by_horizon_sorted = std_by_horizon.sort_values('normalized_std', ascending=False)

# Crear la gráfica de barras
fig = px.bar(
    std_by_horizon_sorted,
    x='SUBBRAND',
    y='normalized_std',
    title='Error Absoluto Normalizado por Marca (Horizonte 0)',
    labels={'SUBBRAND': 'Marca', 'normalized_std': 'Error Absoluto Normalizado'},
)

# Mostrar la gráfica
fig.show()

La precisión de las predicciones para España se evaluó utilizando el **Mean Absolute Error (MAE)** como métrica principal. Sin embargo, dado que el volumen de datos varía entre sub-marcas, se optó por normalizar los errores utilizando los valores reales de cada sub-marca. Esto plantea ciertos desafíos, especialmente debido a la limitación del análisis a un periodo corto, reducido solo a 2023.

**Conclusiones**

- **Precisión por Sub-marca**: 7up resultó ser la sub-marca con menor precisión en sus predicciones, seguida de Pepsi. Esto es atribuible en parte a que ambas presentan mayores volúmenes de ventas y mayor variabilidad, lo que dificulta la precisión.
- **Pepsi Max**: A pesar de tener el mayor número de ventas, con un promedio alrededor de 120k, Pepsi Max logró mantener un error medio relativamente bajo, cercano a los 10k, lo cual es bastante aceptable.
- **Consideraciones Finales**: Con un mayor volumen de datos y un horizonte temporal más amplio, es probable que la precisión de las predicciones mejore aún más. Sin embargo, con los datos actuales, los resultados son bastante satisfactorios y proporcionan una base sólida para el análisis.